### Quick data overview

Read the `driver_stats.parquet` file. In this demo, we will treat the records on the parquet files as a stream of continually updats for serveral entities, meaning an entity could have been updated more than once .

In [2]:
import pandas as pd
df = pd.read_parquet("./data/driver_stats.parquet")
df

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2022-05-24 00:00:00+00:00,1005,0.188643,0.299873,60,2022-06-08 00:16:19.217
1,2022-05-24 01:00:00+00:00,1005,0.299228,0.973971,251,2022-06-08 00:16:19.217
2,2022-05-24 02:00:00+00:00,1005,0.819493,0.902332,738,2022-06-08 00:16:19.217
3,2022-05-24 03:00:00+00:00,1005,0.285189,0.054842,890,2022-06-08 00:16:19.217
4,2022-05-24 04:00:00+00:00,1005,0.794112,0.015872,290,2022-06-08 00:16:19.217
...,...,...,...,...,...,...
1802,2022-06-07 22:00:00+00:00,1001,0.182367,0.882672,139,2022-06-08 00:16:19.217
1803,2022-06-07 23:00:00+00:00,1001,0.564570,0.656329,303,2022-06-08 00:16:19.217
1804,2021-04-12 07:00:00+00:00,1001,0.393999,0.569553,306,2022-06-08 00:16:19.217
1805,2022-05-31 12:00:00+00:00,1003,0.779023,0.771703,195,2022-06-08 00:16:19.217


Find the unique entities

In [8]:
df.driver_id.unique()

array([1005, 1004, 1003, 1002, 1001])

For each entity, find the latest time stamp it was updated. Those records are the only ones that will be present in the online feature store (Redis)

In [6]:
frames = []
for driver_id in df.driver_id.unique():
    driver_df = df[ df.driver_id == driver_id]
    driver_df = driver_df[driver_df.event_timestamp == driver_df.event_timestamp.max()]
    frames.append(driver_df)
pd.concat(frames)

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
359,2022-06-07 23:00:00+00:00,1005,0.077893,0.667749,606,2022-06-08 00:16:19.217
720,2022-06-07 23:00:00+00:00,1004,0.402548,0.208550,832,2022-06-08 00:16:19.217
1081,2022-06-07 23:00:00+00:00,1003,0.497769,0.252555,596,2022-06-08 00:16:19.217
1442,2022-06-07 23:00:00+00:00,1002,0.382991,0.181249,787,2022-06-08 00:16:19.217
1803,2022-06-07 23:00:00+00:00,1001,0.564570,0.656329,303,2022-06-08 00:16:19.217


In [9]:
df.loc[df['driver_id'] == 1005, 'driver_id'] = 10

In [11]:
df.to_parquet("./data/driver_stats.parquet")